# KPIs Basiques - Test de Qualité des Données

**Auteur** : Short Kings Team  
**Date** : 17/12/2024

## Objectifs
Ce notebook permet de :
1. **Valider la qualité des données** (valeurs manquantes, cohérence)
2. **Afficher des KPIs basiques** de la Coupe du Monde

# 1. Setup et Connexion

In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()

# Connexion à la base de données
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

# Configuration des visualisations
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 10

print("Connexion établie avec succès!")

Connexion établie avec succès!


In [2]:
def run_query(query: str) -> pd.DataFrame:
    """Exécute une requête SQL et retourne un DataFrame."""
    with engine.connect() as conn:
        return pd.read_sql(text(query), conn)

# 2. Validation de la Qualité des Données

## 2.1 Vérification des données manquantes -> choix de ne pas encherir les matchs preliminaires

In [3]:
query_nulls = """
SELECT
    COUNT(*) AS total_matchs,
    COUNT(*) - COUNT(date) AS dates_manquantes,
    COUNT(*) - COUNT(city) AS villes_manquantes,
    COUNT(*) - COUNT(round) AS rounds_manquants,
    COUNT(*) - COUNT(home_result) AS scores_home_manquants,
    COUNT(*) - COUNT(away_result) AS scores_away_manquants
FROM matches
"""

df_nulls = run_query(query_nulls)
print("Vérification des valeurs manquantes :")
df_nulls

Vérification des valeurs manquantes :


,total_matchs,dates_manquantes,villes_manquantes,rounds_manquants,scores_home_manquants,scores_away_manquants
0,7446,6480,0,0,0,0


## 2.2 Cohérence des scores -> Données valides, L'Australie s'amuse contre les petites Iles

In [4]:
query_scores = """
SELECT 
    SUM(CASE WHEN home_result < 0 THEN 1 ELSE 0 END) AS scores_home_negatifs,
    SUM(CASE WHEN away_result < 0 THEN 1 ELSE 0 END) AS scores_away_negatifs,
    MIN(home_result) AS min_score_home,
    MAX(home_result) AS max_score_home,
    MIN(away_result) AS min_score_away,
    MAX(away_result) AS max_score_away
FROM matches
"""

df_scores = run_query(query_scores)
print("Cohérence des scores :")
df_scores

Cohérence des scores :


,scores_home_negatifs,scores_away_negatifs,min_score_home,max_score_home,min_score_away,max_score_away
0,0,0,0,31,0,22


## 2.3 Nombre de matchs par édition -> pertes de donnée dûr à mauvais mapping de round

In [6]:
query_editions = """
SELECT 
    edition,
    COUNT(*) AS nb_matchs
FROM matches
GROUP BY edition
ORDER BY edition
"""

df_editions = run_query(query_editions)
print(f"Nombre d'éditions : {len(df_editions)}")
df_editions

Nombre d'éditions : 22


,edition,nb_matchs
0,1930,18
1,1934,51
2,1938,49
3,1950,51
4,1954,83
5,1958,125
6,1962,124
7,1966,159
8,1970,204
9,1974,263


# 3. KPIs Footballistiques

## 3.1 Champions du Monde (victoires en finale) -> problème de données : après investigation mauvaise gestion des draw en phase eliminatoire

In [7]:
query_champions = """
SELECT t.nom_standard AS equipe, COUNT(*) AS titres
FROM matches m
JOIN teams t ON (
    (m.result = 'home_team' AND m.home_team_id = t.id_team) OR
    (m.result = 'away_team' AND m.away_team_id = t.id_team)
)
WHERE m.round = 'Final'
GROUP BY t.nom_standard
ORDER BY titres DESC
"""

df_champions = run_query(query_champions)
print("Champions du Monde :")
df_champions

Champions du Monde :


,equipe,titres
0,Germany,4
1,Brazil,4
2,Argentina,3
3,Italy,3
4,Uruguay,2
5,France,2
6,England,1
7,Spain,1


## 3.2 Finalistes (participations en finale)

In [9]:
query_finalistes = """
SELECT t.nom_standard AS equipe, COUNT(*) AS finales
FROM matches m
JOIN teams t ON t.id_team IN (m.home_team_id, m.away_team_id)
WHERE m.round = 'Final'
GROUP BY t.nom_standard
ORDER BY finales DESC
"""

df_finalistes = run_query(query_finalistes)
print("Équipes ayant disputé le plus de finales :")
df_finalistes

Équipes ayant disputé le plus de finales :


,equipe,finales
0,Germany,8
1,Brazil,7
2,Italy,6
3,Argentina,6
4,France,4
5,Netherlands,3
6,Uruguay,2
7,Hungary,2
8,Czechoslovakia,2
9,Sweden,1


## 3.3 Top équipes par nombre de matchs joués en phase final


In [14]:
query_matchs = """
SELECT t.nom_standard AS equipe, COUNT(*) AS matchs
FROM teams t
JOIN matches m ON t.id_team IN (m.home_team_id, m.away_team_id)
WHERE m.round != 'Preliminary'
GROUP BY t.nom_standard
ORDER BY matchs DESC
LIMIT 15
"""

df_matchs = run_query(query_matchs)
print("Top 15 équipes par nombre de matchs :")
df_matchs

Top 15 équipes par nombre de matchs :


,equipe,matchs
0,Brazil,114
1,Germany,112
2,Argentina,88
3,Italy,84
4,France,74
5,England,74
6,Spain,67
7,Mexico,60
8,Uruguay,59
9,Netherlands,55


## 3.4 Top équipes par victoires totales

In [12]:
query_victoires = """
SELECT t.nom_standard AS equipe, COUNT(*) AS victoires
FROM matches m
JOIN teams t ON (
    (m.result = 'home_team' AND m.home_team_id = t.id_team) OR
    (m.result = 'away_team' AND m.away_team_id = t.id_team)
)
GROUP BY t.nom_standard
ORDER BY victoires DESC
LIMIT 15
"""

df_victoires = run_query(query_victoires)
print("Top 15 équipes par nombre de victoires :")
df_victoires

Top 15 équipes par nombre de victoires :


,equipe,victoires
0,Germany,135
1,Brazil,133
2,Mexico,119
3,Argentina,117
4,Italy,114
5,Netherlands,106
6,Spain,104
7,England,101
8,France,98
9,Sweden,95


## 3.5 Évolution du nombre de buts par édition

In [13]:
query_buts = """
SELECT 
    edition,
    COUNT(*) AS nb_matchs,
    SUM(home_result + away_result) AS total_buts,
    ROUND(AVG(home_result + away_result), 2) AS moy_buts_match
FROM matches
WHERE round != 'Preliminary'
GROUP BY edition
ORDER BY edition
"""

df_buts = run_query(query_buts)
print("Évolution des buts par édition :")
df_buts

Évolution des buts par édition :


,edition,nb_matchs,total_buts,moy_buts_match
0,1930,18,70,3.89
1,1934,17,70,4.12
2,1938,18,84,4.67
3,1950,23,93,4.04
4,1954,26,132,5.08
5,1958,35,126,3.60
6,1962,32,89,2.78
7,1966,32,89,2.78
8,1970,32,95,2.97
9,1974,38,97,2.55


## 3.6 Distribution des résultats

In [16]:
query_results = """
SELECT 
    result,
    COUNT(*) AS nb_matchs,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) AS pourcentage
FROM matches
GROUP BY result
"""

df_results = run_query(query_results)
print("Distribution des résultats :")
df_results

Distribution des résultats :


,result,nb_matchs,pourcentage
0,home_team,3916,52.6
1,draw,1588,21.3
2,away_team,1942,26.1


## 3.7 Top villes hôtes

In [18]:
query_villes = """
SELECT 
    city AS ville,
    COUNT(*) AS nb_matchs
FROM matches
WHERE city IS NOT NULL
GROUP BY city
ORDER BY nb_matchs DESC
LIMIT 15
"""

df_villes = run_query(query_villes)
print("Top 15 villes hôtes :")
df_villes

Top 15 villes hôtes :


,ville,nb_matchs
0,Doha,110
1,Montevideo,86
2,México DF,77
3,San José,74
4,Santiago,71
5,Seoul,71
6,Buenos Aires,68
7,Asunción,68
8,La Paz,64
9,Tehran,64


## 3.8 Matchs avec prolongations / tirs au but

In [19]:
query_extras = """
SELECT
    COUNT(*) AS total_matchs,
    SUM(CASE WHEN extra_time THEN 1 ELSE 0 END) AS prolongations,
    SUM(CASE WHEN penalties THEN 1 ELSE 0 END) AS tirs_au_but,
    ROUND(100.0 * SUM(CASE WHEN extra_time THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_prolongations
FROM matches
"""

df_extras = run_query(query_extras)
print("Statistiques prolongations et tirs au but :")
df_extras

Statistiques prolongations et tirs au but :


,total_matchs,prolongations,tirs_au_but,pct_prolongations
0,7446,52,30,0.7


## 3.9 Stats par confédération

In [20]:
query_conf = """
SELECT 
    t.confederation,
    COUNT(DISTINCT t.id_team) AS nb_equipes,
    COUNT(*) AS matchs_joues
FROM teams t
JOIN matches m ON t.id_team IN (m.home_team_id, m.away_team_id)
WHERE t.confederation IS NOT NULL
GROUP BY t.confederation
ORDER BY matchs_joues DESC
"""

df_conf = run_query(query_conf)
print("Statistiques par confédération :")
df_conf

Statistiques par confédération :


,confederation,nb_equipes,matchs_joues
0,UEFA,60,5890
1,AFC,49,2459
2,CAF,57,2454
3,CONCACAF,36,2074
4,CONMEBOL,11,1651
5,OFC,13,364


## 3.10 Scores extrêmes

In [21]:
query_gros_scores = """
SELECT 
    t1.nom_standard AS home,
    t2.nom_standard AS away,
    m.home_result,
    m.away_result,
    (m.home_result + m.away_result) AS total_buts,
    m.edition,
    m.round
FROM matches m
JOIN teams t1 ON m.home_team_id = t1.id_team
JOIN teams t2 ON m.away_team_id = t2.id_team
ORDER BY (m.home_result + m.away_result) DESC
LIMIT 10
"""

df_gros_scores = run_query(query_gros_scores)
print("Top 10 des matchs avec le plus de buts :")
df_gros_scores

Top 10 des matchs avec le plus de buts :


,home,away,home_result,away_result,total_buts,edition,round
0,Australia,American Samoa,31,0,31,2002,Preliminary
1,Tonga,Australia,0,22,22,2002,Preliminary
2,IR Iran,Guam,19,0,19,2002,Preliminary
3,Maldives,IR Iran,0,17,17,1998,Preliminary
4,Fiji,Tuvalu,16,0,16,2010,Preliminary
5,Tajikistan,Guam,16,0,16,2002,Preliminary
6,American Samoa,Vanuatu,0,15,15,2010,Preliminary
7,Honduras,St. Vincent / Grenadines,11,3,14,1998,Preliminary
8,Fiji,American Samoa,13,0,13,2002,Preliminary
9,New Zealand,Fiji,13,0,13,1982,Preliminary


In [22]:
query_diff_buts = """
SELECT 
    t1.nom_standard AS home,
    t2.nom_standard AS away,
    m.home_result,
    m.away_result,
    ABS(m.home_result - m.away_result) AS difference,
    m.edition,
    m.round
FROM matches m
JOIN teams t1 ON m.home_team_id = t1.id_team
JOIN teams t2 ON m.away_team_id = t2.id_team
ORDER BY ABS(m.home_result - m.away_result) DESC
LIMIT 10
"""

df_diff_buts = run_query(query_diff_buts)
print("Top 10 des plus grandes différences de buts :")
df_diff_buts

Top 10 des plus grandes différences de buts :


,home,away,home_result,away_result,difference,edition,round
0,Australia,American Samoa,31,0,31,2002,Preliminary
1,Tonga,Australia,0,22,22,2002,Preliminary
2,IR Iran,Guam,19,0,19,2002,Preliminary
3,Maldives,IR Iran,0,17,17,1998,Preliminary
4,Tajikistan,Guam,16,0,16,2002,Preliminary
5,Fiji,Tuvalu,16,0,16,2010,Preliminary
6,American Samoa,Vanuatu,0,15,15,2010,Preliminary
7,Fiji,American Samoa,13,0,13,2002,Preliminary
8,New Zealand,Fiji,13,0,13,1982,Preliminary
9,Australia,Solomon Islands,13,0,13,1998,Preliminary


# 4. Résumé

Ce notebook a permis de valider :
- La qualité des données (valeurs manquantes, cohérence des scores)
- Les KPIs de base (champions, finalistes, top équipes)
- Les tendances (évolution des buts, distribution des résultats)
- Les records (scores extrêmes)